In [ ]:
import pandas as pd
import sqlite3
import sqlparse
import os
from groq import Groq

groq_api_key = ""
client = Groq(api_key=groq_api_key)

In [2]:
DB_FILE = "data_2010_temp.db"

df = pd.read_csv("data_2010.csv", sep=';')
meta_sheets = pd.read_excel("sorular-kodlama.xlsx", sheet_name=None)

/var/folders/mz/gzyhrjkn1jd9t0hfvp2g3j_m0000gn/T/ipykernel_15872/2103311230.py:3: DtypeWarning: Columns (6,7,11,12,15,17,18,19,32,45,60,83,84,85,88,89,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,153,182,184,186,208,209,210,211,212,213,214,215,216,217,218,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data_2010.csv", sep=';')


In [3]:
sorular_df = meta_sheets["sorular"]
column_descriptions = []
for i, (_, row) in enumerate(sorular_df.iterrows()):
    if i >= 5:
        break
    col = str(row["Variable"]).strip()
    label = str(row["Label"]).strip()
    measure = str(row["Measurement Level"]).strip()
    column_descriptions.append(f"- {col}: {label} ({measure})")

kodlama_df = meta_sheets["kodlama"]
code_mappings = {}
for _, row in kodlama_df.iterrows():
    var = str(row["Value"]).strip()
    code = str(row["x"]).strip()
    label = str(row["Label"]).strip()
    if var not in code_mappings:
        code_mappings[var] = []
    if len(code_mappings[var]) < 2:
        code_mappings[var].append(f"{code} → {label}")

metadata_text = "DATA COLUMNS:\n" + "\n".join(column_descriptions) + "\n\nCODE MAPPINGS:\n"
for var, mappings in code_mappings.items():
    metadata_text += f"{var}:\n" + "\n".join(mappings) + "\n...\n"

In [4]:
if not os.path.exists(DB_FILE):
    conn = sqlite3.connect(DB_FILE)
    df.to_sql("data_2010", conn, index=False, if_exists="replace")
    conn.close()

In [5]:
def clean_sql_query(raw_sql):
    if raw_sql.startswith("```") and raw_sql.endswith("```"):
        raw_sql = raw_sql.strip("`").strip()
    raw_sql = raw_sql.replace('COUNT(*)"', 'COUNT(*)').replace('COUNT(*) "', 'COUNT(*) ')
    formatted_sql = sqlparse.format(raw_sql, reindent=True, keyword_case='upper')
    return formatted_sql.strip()

In [6]:
def generate_sql(question: str):
    prompt_sql = f"""
    You are a data analyst. Given a user question and a table called 'data_2010', write a correct SQLite SQL query.
    Always wrap column names in double quotes — especially those containing periods or special characters.

    User question: "{question}"

    Here is documentation about the data columns and values:
    {metadata_text}

    Only return the SQL query, no explanation.
    """

    response = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[{"role": "user", "content": prompt_sql}],
        temperature=0.2
    )
    raw_sql_query = response.choices[0].message.content.strip()
    return clean_sql_query(raw_sql_query)

In [7]:
def execute_sql(sql_query: str):
    conn = sqlite3.connect(DB_FILE)
    try:
        result_df = pd.read_sql_query(sql_query, conn)
    except Exception as e:
        conn.close()
        raise e
    conn.close()
    return result_df

In [8]:
def summarize_answer(question: str, result_df: pd.DataFrame):
    top_row = result_df.iloc[0] if not result_df.empty else None
    if top_row is not None:
        summary_text = f"Top row summary:\n{top_row.to_dict()}"
    else:
        summary_text = "No results found."

    prompt_answer = f"""
    You are a data assistant. Based on the following user question and query result summary, provide a short and accurate answer.

    User question: "{question}"

    Query result summary:
    {summary_text}
    """

    response = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[{"role": "user", "content": prompt_answer}],
        temperature=0.3
    )
    return response.choices[0].message.content.strip()

In [9]:
def suggest_questions():
    prompt_suggest = f"""
    You are a data analyst. Given the following metadata, suggest 10 insightful and diverse questions that a user might ask to summarize or understand the dataset 'data_2010'. 
    Focus on providing questions related to counts, distributions, averages, trends, or interesting breakdowns.

    Metadata:
    {metadata_text}

    Only return the 10 questions in a numbered list, no explanations.
    """

    response = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[{"role": "user", "content": prompt_suggest}],
        temperature=0.3
    )
    return response.choices[0].message.content.strip()

In [12]:
question = "Show the number of people per profession."
sql = generate_sql(question)
print("Generated SQL:\n", sql)

df_result = execute_sql(sql)
print("Query Result:\n", df_result)

summary = summarize_answer(question, df_result)
print("Summary:\n", summary)


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01jw10264ket6vmges72m5hq8x` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 497355, Requested 4322. Please try again in 4m49.716199999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}